In [2]:
import networkx as nx
import numpy as np
import os
import io
import json
from multiprocessing import Process
import multiprocessing

In [3]:
# algorithm inspired by https://github.com/jeroenvldj/bow-tie_detection/blob/master/bow-tie_detection.py

def bowtie_analysis(G):
    # reverse all direction of the graph
    GT = nx.reverse(G, copy=True)
    # calculate SSC
    SSC = max(list(nx.strongly_connected_components(G)),key=len)    
    
    
    # take any node n from SSC and do a depth first search 
    # through directed graph beginning from node n
    v_any = list(SSC)[0]
    DFS_G = set(nx.dfs_tree(G,v_any).nodes())
    DFS_GT = set(nx.dfs_tree(GT,v_any).nodes())
    OUT = DFS_G - SSC
    IN = DFS_GT - SSC
    V_rest = set(G.nodes()) - SSC - OUT - IN

    TUBES = set()
    INTENDRILS = set()
    OUTTENDRILS = set()
    OTHER = set()

    for v in V_rest:
        # irv => in reaches node v
        irv = len(IN & set(nx.dfs_tree(GT,v).nodes())) is not 0
        # vro => node v reaches out
        vro = len(OUT & set(nx.dfs_tree(G,v).nodes())) is not 0
        if irv and vro:
            TUBES.add(v)
        elif irv and not vro:
            INTENDRILS.add(v)
        elif not irv and vro:
            OUTTENDRILS.add(v)
        elif not irv and not vro:
            OTHER.add(v)

    FRINGE = set()
    DISCONNECTED = set()
    for o in OTHER:
        # orIT => node o reaches INTENDRILS  
        orIT = len(INTENDRILS & set(nx.dfs_tree(G,o))) is not 0
        # OTro => OUTTERNDIRLS reaches node o
        OTro = len(OUTTENDRILS & set(nx.dfs_tree(GT,o))) is not 0
        if orIT or OTro:
            FRINGE.add(o)
        else:
            DISCONNECTED.add(o)
    
    TENDRILS = INTENDRILS.union(OUTTENDRILS)
    
    def component_result(name, graph_nodes):
        return{ name        : len(graph_nodes),
                name + "_s" : sum([G.nodes()[node]["is_sink"] for node in graph_nodes]),
                name + "_m" : sum([G.nodes()[node]["is_miner"] for node in graph_nodes])}

    result_dict = dict()
    result_dict.update(component_result("nodes", G.nodes()))
    result_dict.update(component_result("ssc", SSC))
    result_dict.update(component_result("in", IN))
    result_dict.update(component_result("out",OUT))
    result_dict.update(component_result("tubes", TUBES))
    result_dict.update(component_result("tendrils", TENDRILS))
    result_dict.update(component_result("fringe", FRINGE))
    result_dict.update(component_result("disconnected", DISCONNECTED))
    
    return result_dict

In [18]:
def randomize(g):
    graph = g.copy()
    undirected_graph = graph.to_undirected()
    nswap=len(undirected_graph.edges())/2
    max_tries=100*len(undirected_graph.edges())
    seed = np.random.seed()
    nx.algorithms.swap.double_edge_swap(undirected_graph, nswap=nswap,seed=seed, max_tries=max_tries)
    return undirected_graph.to_directed()


In [8]:
g = nx.read_graphml("../data/NET-btc-heur_0-week/2010-10-18.graphml")
g2 = g.copy()

In [6]:
d1 = bowtie_analysis(g)

In [23]:
g2 = g.copy()
d2 = bowtie_analysis(randomize(g2.copy()))
d2

{'nodes': 5198,
 'nodes_s': 88,
 'nodes_m': 0,
 'ssc': 4951,
 'ssc_s': 77,
 'ssc_m': 0,
 'in': 0,
 'in_s': 0,
 'in_m': 0,
 'out': 0,
 'out_s': 0,
 'out_m': 0,
 'tubes': 0,
 'tubes_s': 0,
 'tubes_m': 0,
 'tendrils': 0,
 'tendrils_s': 0,
 'tendrils_m': 0,
 'fringe': 0,
 'fringe_s': 0,
 'fringe_m': 0,
 'disconnected': 247,
 'disconnected_s': 11,
 'disconnected_m': 0}

In [20]:
d1

{'nodes': 5198,
 'nodes_s': 88,
 'nodes_m': 0,
 'ssc': 265,
 'ssc_s': 0,
 'ssc_m': 0,
 'in': 746,
 'in_s': 0,
 'in_m': 0,
 'out': 974,
 'out_s': 2,
 'out_m': 0,
 'tubes': 695,
 'tubes_s': 0,
 'tubes_m': 0,
 'tendrils': 1589,
 'tendrils_s': 82,
 'tendrils_m': 0,
 'fringe': 253,
 'fringe_s': 1,
 'fringe_m': 0,
 'disconnected': 676,
 'disconnected_s': 3,
 'disconnected_m': 0}

In [20]:
d2

{'nodes': 5198,
 'nodes_s': 88,
 'nodes_m': 0,
 'ssc': 4907,
 'ssc_s': 73,
 'ssc_m': 0,
 'in': 0,
 'in_s': 0,
 'in_m': 0,
 'out': 0,
 'out_s': 0,
 'out_m': 0,
 'tubes': 0,
 'tubes_s': 0,
 'tubes_m': 0,
 'tendrils': 0,
 'tendrils_s': 0,
 'tendrils_m': 0,
 'fringe': 0,
 'fringe_s': 0,
 'fringe_m': 0,
 'disconnected': 291,
 'disconnected_s': 15,
 'disconnected_m': 0}

In [4]:
def files_walker(files, json_output,directory):
    #files = os.listdir(directory)
    #files = directory
    for file in files:
        with open(directory+'/'+file, 'r', encoding='utf8', errors='ignore') as f:
            G = nx.read_graphml(f)
            bowtie_dict = bowtie_analysis(G)
            with open(json_output, "r+") as fi:
                data = json.load(fi)
                new_entry = {file[:-8] : bowtie_dict}
                data.update(new_entry)
                fi.seek(0)
                json.dump(data, fi, indent=4)
    return True

In [11]:
import time
start_time = time.time()


nr_cores = multiprocessing.cpu_count()

core_dict = {}
np_files = np.array(os.listdir("../data/NET-btc-heur_0-week"))
directory = "data/NET-btc-heur_0-week"
chunk_lst = np.array_split(np_files, nr_cores)

for i in range(nr_cores):
    process_name = f"p{i}"
    file_name = f"test{i}.json"

    with io.open(os.path.join(file_name), 'w') as db_file:
        db_file.write(json.dumps({}))

    core_dict[process_name] = Process(target = files_walker, args= (list(chunk_lst[i]),file_name, directory))
    
    core_dict[process_name].start()

    core_dict[process_name].join()

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.1553215980529785 seconds ---
